| | 。。。。。。。。。。。。。。。方法。。。。。。。。。。。。。。。 | 适用于 |
| --- | --- | --- |
| 特征选择1 | $ G_k=\frac{(m_{ik}-m_{jk})^2}{\sigma_{ik}^2+\sigma_{jk}^2},G_k大则优先 $ | 正态或近似正态分布（有重叠需要再加其他特征） |
| 特征选择2 | $ J_1=det(S_w^{-1}S_b)=\Pi_i \lambda_i=J_2=tr(S_w^{-1}S_b),\lambda_i特征值 $ | 适用于任何分布 |
| 特征提取（K-L,PCA) | $ a=\Phi[:,:m]^T x',\Phi是x'的自相关矩阵的特征向量，x'=x-m $ | 适用于任何分布，当E(x)!=0的时候，只能达到次最优，最好先标准化 |

* 特征选择和提取
    * 组合计算特征，数据多维度加权-->
    * 不作处理：维度诅咒
    * 要求:确保精度下，降维
    * 如何选择？
        1. 去除不易判别的特征
        2. 去除相关性强的特征（重复）
    * 分类
        * 特征选择：选择（x还是x，但选了几个出来
            * 选择依据：模式类别可分性
        * 特征提取：x->y=f(x)变换到y(二次特征）

* 选择依据：模式类别可分性的测度
    * 点与点的距离D(a,b)=||a-b||
    * 点与点集的距离 $ =\frac{1}{K}\Sigma_{i=1}^k D^2(x,a^{(i)}),a^{(i)} \in A点集 $
    * 类内距离：点集内部距离：类内各点的均方距离
        * $ D^2=\frac{1}{K}\Sigma_{i=1}^k(\Sigma_{j=1,j \neq i}^k D^2(a^{(i)},a^{(j)}))=2\Sigma_{k=1}^n \sigma_k^2 $
            * 方差$ \sigma_k^2=\frac{1}{K-1}\Sigma_{i=1}^k(a_k^{(i)}-a_k)^2 $
                * $ a_k=\frac{1}{K}\Sigma_{i=1}^K a_k^{(i)} $
                * 右下：k个分量--k维度
                * 右上：K个a，第几个样本
    * 类内散布矩阵
        * $ S=\Sigma_{i=1}^K(a^{(i)}-m)(a^{(i)}-m)^T $,m-均值
        * 多类模式下集散布矩阵(类内）
            *  $ S_w=\Sigma_{i=1}^cp(\omega_i)E\{(x-m_i)(x-m_i)^T|\omega_i\}=\Sigma_{i=1}^cp(\omega_i)C_i $---x是i的x
            * $ 协方差矩阵：C_i=E\{(x-m_i)(x-m_i)^T|\omega_i\} $
    * 类间距离：$ S_{b_2}=D^2=||m_1-m_2|| =(m_1-m_2)(m_1-m_2)^T $
        * 三个类别以上：$ S_b=\Sigma_{i=1}^cp(\omega_i)(m_i-m_0)(m_i-m_0)^T $
            * m0:总体均值$ m_0=E(x)=\Sigma_{i=1}^cp(\omega_i)m_i $
    
    * 多类模式下总体分布的散布矩阵：
        * $ S_t=E\{(x-m_0)(x-m_0)^T\} $
        * $ S_t=S_w+S_b $

## 特征选择
* n个特征中选择m个特征：$ C_n^m $选法
* 目标：类内距离最小，类间距离最大
* 如何选择
    1. 穷举（太多了
    2. 由可分性准则判定其优劣
        * 独立特征的选择准则
            * 独立地进行分析即可
            * 准则函数（优先级）：$ G_k=\frac{(m_{ik}-m_{jk})^2}{\sigma_{ik}^2+\sigma_{jk}^2},k=1,2,...,n $
            * Gk大的优先
            * 适用范围
                1. 正态分布，完全可分，ok
                2. 正态分布，大多重叠，仅依赖xk不行，需要添加其他特征
                3. 非或不近似正态分布不行：wi有两个峰，不可用
        * 一般特征的选择准则
            * 直观上，Sw类间散布矩阵大，Sb类内散布矩阵小好
                * 行列式形式$ J_1=det(S_w^{-1}S_b)=\Pi_i \lambda_i $
                * 迹形式$ J_2=tr(S_w^{-1}S_b)=\Pi_i \lambda_i,\lambda_i特征值 $
                * 选择使$ J_1或J_2 $ 最大的子集

In [27]:
import numpy as np
x1=np.array([[1,2,1],[0,0,1]])
x2=np.array([[-1,0,-1],[0,1,1]])
x3=np.array([[-1,0,0],[-1,-1,-2]])
m1=np.mean(x1,axis=1).reshape(2,1)
m2=np.mean(x2,axis=1).reshape(2,1)
m3=np.mean(x3,axis=1).reshape(2,1)
print(3*m1)

Sw=(np.matmul((x1-m1),(x1-m1).T)+np.matmul((x2-m2),(x2-m2).T)+np.matmul((x3-m3),(x3-m3).T))/3
print(Sw)
m0=(m1+m2+m3)/3
print(m0)

Sb=(np.matmul((m1-m0),(m1-m0).T)+np.matmul((m2-m0),(m2-m0).T)+np.matmul((m3-m0),(m3-m0).T))/3
print(81*Sb)


[[4.]
 [1.]]
[[ 0.66666667 -0.11111111]
 [-0.11111111  0.66666667]]
[[ 0.11111111]
 [-0.11111111]]
[[62. 13.]
 [13. 62.]]


## 特征提取--PCA--离散K-L变换
* 适用于任何概率密度函数
* x--正交变换(线性组合)-->a,再从a中选择前几个
* 有限的K-L:
    * $ x_i=\Phi a_i ,正交变换矩阵\Phi第t行j列\phi_j(t)，x->a  $
    * 展开：$  x_i(t)=\Sigma_{i=1}^{\infty}\phi_j(t) a_j,T_1<a_j<T_2;\phi_j连续正交，a_j展开的随即系数 $
    * 总体自相关矩阵$ R=E\{xx^T\} $
        * $ R=E\{xx^T\}=E\{\Phi aa^T\Phi^T\}=\Phi E\{ aa^T\}\Phi^T $
        * 令$  E\{ aa^T\}=deg({\lambda_i})=D_\lambda（a_ia_i=\lambda_i,其他0 $ ——特征值的对角矩阵
        * $ R=\Phi D_\lambda \Phi^T ，\Phi--特征矩阵 $
        * $ \Phi R=\Phi D_\lambda $
        * $ R \phi_j=\lambda_j \phi_j $
    * ==>$ a=\Phi^T x $
    * ==>$ a=\Phi[:,0:m]^T x $ ,取前m个特征向量（对应特征值从大到小排序）
        * 证明，可这么做：
            * $ x'=\Sigma_{j=1}^ma_j\phi_j+\Sigma_{j=1+m}^nb\phi_j,b是常数，得到x的估计值x' $
            * $ \Delta x=x-x'=\Sigma_{j=1+m}^n(a_j-b)\phi_j $
            * $ \epsilon^2=E\{||\Delta x||^2\}= \Sigma_{j=1+m}^nE((a_j-b)^2) $
            * $ 为了让\epsilon^2最小，求导： $
            * $ \frac{\partial }{\partial b}{E((a_j-b)^2)}=\frac{\partial }{\partial b}{E(a_j^2-2a_jb+b^2)}=-2E(a_j)-2b=0 $
            * $ b=E(a_j) $
            * $ \epsilon^2=\Sigma_{j=1+m}^nE((a_j-E(a_j))^2)=\Sigma_{j=1+m}^n\phi_j^TE((x-E(x))(x-E(x))^T)\phi_j=\Sigma_{j=1+m}^n\phi_j^TC_x\phi_j $
                * $ c_x\phi_j=\lambda_j \phi_j ;\lambda_j为C_x的特征值，\phi_j是特征向量 $
                * $ \phi_j^T\phi_j=1==>\phi_j^Tc_x\phi_j=\lambda_j  $
            * $ \epsilon^2=\Sigma_{j=1+m}^n\phi_j^TC_x\phi_j=\Sigma_{j=1+m}^n\lambda_j  $
            * 所以，当$ \lambda_j越小$，误差就越小
    * 选择原则，按最小方差原则选择$ E(a_j)=0==>E(x)=0,若不等于0，只能得到次最佳结果 $（标准化后结果更好）
    * 选取协方差矩阵C或自相关矩阵R来计算特征值/特征向量
        * $ C=E((x-E(x))(x-E(x))^T)=E(xx^T)-2E(x)^2=R-2m^2 $
    * 所获取的a是**互不相关**的新特征
    * 变换步骤：（x是列向量）
        1. 求均值m(整体的均值)（x'=x-m，使得m=0)
        2. 求自相关矩阵R
        3. 求R的特征值$ \lambda $
        4. 求特征值对应的特征向量：$ \phi $，单位向量，且正交
        5. 取前k个，$ a=\Phi[:,:m] x'$

In [48]:
import numpy as np
x1=np.array([[0,2,2,1],[0,0,0,2],[0,0,1,0]])
x2=np.array([[0,0,0,1],[0,1,-2,1],[1,0,1,-2]])
x=np.array([[0,2,2,1,0,0,0,1],[0,0,0,2,0,1,-2,1],[0,0,1,0,1,0,1,-2]])
m1=np.mean(x1,axis=1).reshape(3,1)
m2=np.mean(x2,axis=1).reshape(3,1)
m=(m1+m2)/2
print(m)
C1=np.matmul((x1-m1),(x1-m1).T)/4
C2=np.matmul((x2-m2),(x2-m2).T)/4
C=(C1+C2)/2#这个应该不行，类内离散度
print(C)
C_=np.matmul((x-m),(x-m).T)/8
print(C_)

#这两种R是一样的，m=0时才用
R1=np.matmul(x1,x1.T)/4
R2=np.matmul(x2,x2.T)/4
R=(R1+R2)/2
print(R)
R_=np.matmul(x,x.T)/8
print(R_)

[[0.75 ]
 [0.25 ]
 [0.125]]
[[ 0.4375   0.0625  -0.15625]
 [ 0.0625   1.125   -0.5625 ]
 [-0.15625 -0.5625   0.84375]]
[[ 0.6875    0.1875   -0.09375 ]
 [ 0.1875    1.1875   -0.53125 ]
 [-0.09375  -0.53125   0.859375]]
[[ 1.25   0.375  0.   ]
 [ 0.375  1.25  -0.5  ]
 [ 0.    -0.5    0.875]]
[[ 1.25   0.375  0.   ]
 [ 0.375  1.25  -0.5  ]
 [ 0.    -0.5    0.875]]


In [49]:
print('C=C1+C2')
lamb,vec=np.linalg.eig(C)
print(np.linalg.eig(C))
print('C_')
lamb_,vec_=np.linalg.eig(C)
print(np.linalg.eig(C_))
print('R')
print(np.linalg.eig(R))

C=C1+C2
(array([1.5827506 , 0.49608042, 0.32741898]), array([[-0.12662322, -0.73030067,  0.67128793],
       [-0.7762265 ,  0.49429774,  0.39133382],
       [ 0.61760746,  0.47151953,  0.62946831]]))
C_
(array([1.625     , 0.64876246, 0.46061254]), array([[ 0.21538745,  0.95853318, -0.18660756],
       [ 0.78975397, -0.05858624,  0.61061961],
       [-0.57436653,  0.27889386,  0.76962413]]))
R
(array([1.78643783, 1.125     , 0.46356217]), array([[-0.52255387,  0.80178373, -0.28996605],
       [-0.74751377, -0.26726124,  0.60810739],
       [ 0.41007392,  0.53452248,  0.73900277]]))


In [46]:
np.matmul(np.matmul(vec.T,C),vec)# vec,一列一个特征向量

array([[1.58275060e+00, 0.00000000e+00, 2.22044605e-16],
       [5.55111512e-17, 4.96080417e-01, 7.77156117e-16],
       [1.94289029e-16, 7.77156117e-16, 3.27418981e-01]])

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(x1[])

In [58]:
np.matmul(C_,vec_[:,0])

array([-0.29049663, -1.27361479,  0.95499767])

In [65]:
np.matmul(vec_,np.matmul(np.diag(lamb_),vec_.T))

array([[ 0.4375 ,  0.0625 , -0.15625],
       [ 0.0625 ,  1.125  , -0.5625 ],
       [-0.15625, -0.5625 ,  0.84375]])

In [60]:
C_

array([[ 0.6875  ,  0.1875  , -0.09375 ],
       [ 0.1875  ,  1.1875  , -0.53125 ],
       [-0.09375 , -0.53125 ,  0.859375]])

In [67]:
x_11=np.matmul(vec_[:,2].T,x1)
print(x_11)

[0.         1.34257586 1.97204417 1.45395558]


In [68]:
x_21=np.matmul(vec_[:,2].T,x2)
print(x_21)

[ 0.62946831  0.39133382 -0.15319933 -0.19631487]


In [69]:
x_11=np.matmul(vec_[:,2].T,x1-m)
print(x_11)
x_21=np.matmul(vec_[:,2].T,x2-m)
print(x_21)

[-0.67998294  0.66259292  1.29206123  0.77397263]
[-0.05051463 -0.28864912 -0.83318228 -0.87629781]
